In [17]:
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git
!pip install tqdm

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rfe_47fa
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rfe_47fa
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!rm -rf /content/drive/MyDrive/image

In [ ]:
import struct
from struct import unpack
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
import torch

list_image_path = []
list_txt = []

# Upack the drawing data
def unpack_drawing(file_handle):
    key_id, = unpack('Q', file_handle.read(8)) # Read 8 bytes as an unsigned long long
    country_code, = unpack('2s', file_handle.read(2))
    recognized, = unpack('b', file_handle.read(1))
    timestamp, = unpack('I', file_handle.read(4))
    n_strokes, = unpack('H', file_handle.read(2)) # Read 2 bytes as an unsigned short
    image = [] # Store the strokes
    for i in range(n_strokes):
        n_points, = unpack('H', file_handle.read(2))
        fmt = str(n_points) + 'B'
        x = unpack(fmt, file_handle.read(n_points))
        y = unpack(fmt, file_handle.read(n_points))
        #y = tuple(255 - yi for yi in y)  # Invert the image
        image.append((x, y)) # Append the stroke to the image

    return {
        'key_id': key_id,
        'country_code': country_code.decode(),
        'recognized': recognized,
        'timestamp': timestamp,
        'image': image
    }


# Unpack all the drawings in a file
def unpack_drawings(filename):
    with open(filename, 'rb') as f:
      while True:
        try:
          drawing = unpack_drawing(f)
          #print("Unpack_drawings: \n")
          #plot_drawing(drawing['image'])
          yield unpack_drawing(f)
        except struct.error:
          break

# Plot the drawing
def plot_drawing(drawing):
    fig, ax = plt.subplots()
    for x, y in drawing:
        ax.plot(x, y)
    plt.gca().invert_yaxis()
    plt.show()

def save_drawing_as_image(drawing, folder_path, image_index):
    #print("before save_drawing_as_image: \n")
    #plot_drawing(drawing['image'])
    image = Image.new('L', (256, 256), 255) # Create a white image
    draw = ImageDraw.Draw(image)

    try:
      for x, y in drawing['image']:
        if len(x) > 1 and len(y) > 1:
            points = list(zip(x, y))
            draw.line(points, fill = 0, width = 1)
        else:
            for i in range(len(x)):
                if 0 <= x[i] < 256 and 0 <= y[i] < 256:
                    draw.point((x[i], y[i]), fill = 0)
    except:
      pass

    image_path = os.path.join(folder_path, f'image_{image_index}.jpg')
    image.save(image_path)

    # choose the first image of each category
    if image_index == 0:
        list_image_path.append(image_path)
        folder_name = os.path.basename(folder_path)
        list_txt.append(folder_name)
    print(f"Saved image {image_index}.jpg to {folder_path}")



    """
    pixels = image.load()

    for x, y in drawing['image']:
        for i in range(len(x)):
          pixels[x[i], y[i]] = 0 # Set the pixel to black
    image_path = os.path.join(folder_path, f'image_{image_index}.jpg')
    image.save(image_path)

    # choose the first image of each category
    if image_index == 0:
        list_image_path.append(image_path)
        folder_name = os.path.basename(folder_path)
        list_txt.append(folder_name)
    print(f"Saved image {image_index}.jpg to {folder_path}")
    #print("after save_drawing_as_image: \n")
    #plot_drawing(drawing['image'])
"""
def process_bin_files(bin_folder, image_folder, max_images_per_type = 100, num = 1):
    for bin_file in os.listdir(bin_folder):
        if bin_file.endswith('.bin'):
            bin_file_path = os.path.join(bin_folder, bin_file)
            drawing_name = os.path.splitext(bin_file)[0]

            drawing_folder_path = os.path.join(image_folder, drawing_name)
            os.makedirs(drawing_folder_path, exist_ok=True)


            print(f"Processing {bin_file_path}, {num} / 345")
            num += 1

            for image_index, drawing in enumerate(unpack_drawings(bin_file_path)):
                if image_index >= max_images_per_type:
                  break
                save_drawing_as_image(drawing, drawing_folder_path, image_index)

bin_folder = '/content/drive/MyDrive/binary'
image_folder = '/content/drive/MyDrive/image'
os.makedirs(image_folder, exist_ok=True)
#for drawing in unpack_drawings('/content/drive/MyDrive/binary/boomerang.bin'):
    # do something with the drawing
    #print("original: \n")
    #plot_drawing(drawing['image'])
process_bin_files(bin_folder, image_folder)

Processing /content/drive/MyDrive/binary/boomerang.bin, 1 / 345
Saved image 0.jpg to /content/drive/MyDrive/image/boomerang
Saved image 1.jpg to /content/drive/MyDrive/image/boomerang
Saved image 2.jpg to /content/drive/MyDrive/image/boomerang
Saved image 3.jpg to /content/drive/MyDrive/image/boomerang
Saved image 4.jpg to /content/drive/MyDrive/image/boomerang
Saved image 5.jpg to /content/drive/MyDrive/image/boomerang
Saved image 6.jpg to /content/drive/MyDrive/image/boomerang
Saved image 7.jpg to /content/drive/MyDrive/image/boomerang
Saved image 8.jpg to /content/drive/MyDrive/image/boomerang
Saved image 9.jpg to /content/drive/MyDrive/image/boomerang
Saved image 10.jpg to /content/drive/MyDrive/image/boomerang
Saved image 11.jpg to /content/drive/MyDrive/image/boomerang
Saved image 12.jpg to /content/drive/MyDrive/image/boomerang
Saved image 13.jpg to /content/drive/MyDrive/image/boomerang
Saved image 14.jpg to /content/drive/MyDrive/image/boomerang
Saved image 15.jpg to /content/

In [ ]:
print(list_image_path)
print(list_txt)
print(len(list_image_path))
print(len(list_txt))

In [ ]:
import json # Import the json module: handling JSON data
from PIL import Image # Import the image module from the PIL library: image processing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

import clip
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

# Load the model and corresponding preprocessor for image and text
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

class image_title_dataset():
    def __init__(self, list_image_path,list_txt):
        # Initialize image paths and corresponding texts
        self.image_path = list_image_path
        # Tokenize text using CLIP's tokenizer
        self.title  = clip.tokenize(list_txt)

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        # Preprocess image using CLIP's preprocessing function
        image = preprocess(images=Image.open(self.image_path[idx]), return_tensors="pt")["pixel_values"].squeeze(0)
        title = self.title[idx]
        return image, title


dataset = image_title_dataset(list_image_path[:100],list_txt[:100])
BATCH_SIZE = 2 # should greater than 1
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

if device == "cpu":
  model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-6, betas = (0.9, 0.98), eps = 1e-6, weight_decay = 0.2)

num_epochs = 300
for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, total = len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()

        images, texts = batch
        images = images.to(device)
        texts = texts.to(device)

        # Forward pass
        outputs = model(pixel_values=images, input_ids=texts)
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text


         # Check for nan values in logits
        if torch.isnan(logits_per_image).any() or torch.isnan(logits_per_text).any():
            print("Found nan values in logits")
            continue

        # Calculate loss
        ground_truth = torch.arange(len(images), dtype = torch.long, device = device)
        total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth))/2

        # Backward pass
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # ensure no grad explord

        if device == "cuda":
            optimizer.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)
        pbar.set_description(f"EPOCH: {epoch}/{num_epochs}, LOSS: {total_loss.item():.9f}")


In [ ]:
# Save the model
checkpoint_dir = "model_checkpoint"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': total_loss,
    }, f"model_checkpoint/model_10.pt")

In [ ]:
# Load the model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
modeltuning = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

checkpoint = torch.load("model_checkpoint/model_10.pt")
modeltuning.load_state_dict(checkpoint['model_state_dict'])

model.eval()
modeltuning.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
modeltuning.to(device)


In [ ]:
# preprocess the test images and texts
test_image_path = ["/content/ear.jpg", "/content/ball.jpg"]
test_texts = ["ear", "ball"]

# preprocess the image
test_images = [Image.open(img_path).convert("RGB") for img_path in test_image_path]
test_images_preprocessed = torch.cat([preprocess(images=img, return_tensors="pt")["pixel_values"] for img in test_images]).to(device)

# preprocess the text
test_texts_preprocessed = clip.tokenize(test_texts).to(device)




In [ ]:
# without Tuning
with torch.no_grad():
    outputs = model(pixel_values=test_images_preprocessed, input_ids=test_texts_preprocessed)
    logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

# with Tuning
with torch.no_grad():
    outputstuning = modeltuning(pixel_values=test_images_preprocessed, input_ids=test_texts_preprocessed)
    logits_per_imagetuning, logits_per_texttuning = outputstuning.logits_per_image, outputstuning.logits_per_text


def check_success(logits):
    success_count = 0
    for m, image_list in enumerate(logits.tolist()):
        max_similarity = max(image_list)
        for n, similarity in enumerate(image_list):
            #print(f"Similarity between image {m + 1} and description {n + 1}: {similarity}\n")
            if m == n and similarity == max_similarity:
                success_count += 1
    return success_count

print("Before Tuning: \n")
success_count_of_image =check_success(logits_per_image)
success_count_of_text = check_success(logits_per_text)
print(f"\nCount of successful recognitions: {success_count_of_image + success_count_of_text}\n")


print("\nAfter Tuning: \n")
success_count_of_image = check_success(logits_per_imagetuning)
success_count_of_text = check_success(logits_per_texttuning)
print(f"\nCount of successful recognitions: {success_count_of_image + success_count_of_text}\n")